In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoporosis"
cohort = "GSE35925"

# Input paths
in_trait_dir = "../../input/GEO/Osteoporosis"
in_cohort_dir = "../../input/GEO/Osteoporosis/GSE35925"

# Output paths
out_data_file = "../../output/preprocess/Osteoporosis/GSE35925.csv"
out_gene_data_file = "../../output/preprocess/Osteoporosis/gene_data/GSE35925.csv"
out_clinical_data_file = "../../output/preprocess/Osteoporosis/clinical_data/GSE35925.csv"
json_path = "../../output/preprocess/Osteoporosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Calcitriol supplementation effects on Ki67 expression and transcriptional profile of breast cancer specimens from post-menopausal patients"
!Series_summary	"Background: Breast cancer patients present lower 1,25(OH)2D3 or 25(OH)D3 serum levels than unaffected women. Although 1,25(OH)2D3 pharmacological concentrations of 1,25(OH)2D3 may exert antiproliferative effects in breast cancer cell lines, much uncertainty remains about the effects of calcitriol supplementation in tumor specimens in vivo. We have evaluated tumor dimension (ultrassonography), proliferative index (Ki67 expression), 25(OH)D3 serum concentration and gene expression profile, before and after a short term calcitriol supplementation (dose to prevent osteoporosis) to post-menopausal patients. Results: Thirty three patients with operable disease had tumor samples evaluated. Most of them (87.5%) presented 25(OH)D3 insufficiency (<30 ng/mL). Median period of calcitriol supplementation w

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptional analysis (U133 Plus 2.0 GeneChip)
# which suggests gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: From the background information, these are breast cancer samples 
# No direct mention of osteoporosis in patients, only that calcitriol is used at doses to prevent osteoporosis
# Since everyone has breast cancer (tissue type: breast cancer), there's no variation in trait status
trait_row = None  # No variation in trait status for osteoporosis

# For age: Age information is available in row 1
age_row = 1

# For gender: All patients are female as mentioned in row 0
gender_row = 0  # However, this is constant (all female)

# 2.2 Data Type Conversion

# Since trait data is not available for osteoporosis, we create a placeholder function
def convert_trait(value):
    return None

# Age conversion function - ages are numeric values
def convert_age(value):
    try:
        # Extract the value after the colon and convert to float
        return float(value.split(': ')[1].strip())
    except:
        return None

# Gender conversion function - converting female to 0, male to 1
def convert_gender(value):
    try:
        gender = value.split(': ')[1].strip().lower()
        if gender == 'female':
            return 0
        elif gender == 'male':
            return 1
        else:
            return None
    except:
        return None

# 3. Save Metadata
# Trait data is not available for osteoporosis
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (like '1007_s_at', '1053_at'), not human gene symbols.
# Affymetrix probe IDs need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene mapping
# Looking at the data, 'ID' contains identifiers like '1007_s_at' which match gene_data's index
# 'Gene Symbol' contains gene symbols like 'DDR1 /// MIR4640'
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the gene mapping dataframe using the get_gene_mapping function
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Convert probe-level measurements to gene expression data using apply_gene_mapping function
# This will divide probe measurements equally among mapped genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first 20 rows to verify the transformation from probe IDs to gene symbols
print("After mapping, first 20 gene symbols:")
print(gene_data.index[:20])


After mapping, first 20 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06', 'AAAS', 'AACS', 'AACSP1', 'AADAC', 'AADACL2',
       'AADACP1', 'AADAT', 'AAED1', 'AAGAB', 'AAK1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Check if trait_row is None (no trait data available)
if trait_row is None:
    # If no trait data available, mark dataset as unusable for trait study
    print("No trait data available for osteoporosis in this dataset.")
    
    # Use is_final=False for cases where trait data is not available
    validate_and_save_cohort_info(
        is_final=False, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False
    )
    
    print("Dataset determined to be unusable for osteoporosis analysis and linked data was not saved")
else:
    # This block won't be executed for this dataset, but included for completeness
    clinical_features_df = geo_select_clinical_features(
        clinical_data, 
        trait=trait, 
        trait_row=trait_row, 
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
    print("Linked data shape:", linked_data.shape)

    # Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)

    # Determine whether the trait and demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

    # Conduct quality check and save the cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=unbiased_linked_data,
        note="Contains gene expression data from breast cancer patients with trait information."
    )

    # Save linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Osteoporosis/gene_data/GSE35925.csv
No trait data available for osteoporosis in this dataset.
Dataset determined to be unusable for osteoporosis analysis and linked data was not saved
